In [1]:
import cv2
from matplotlib import pyplot as plt
import seaborn as sns
import collections
import numpy as np
import itertools
import glob
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
# from realsense_detector import detect_contour
import copy
# from realsense_detector import main2

%config InlineBackend.figure_format = 'retina'

In [3]:
def count_color(image):
    color_list = []
    for i in range(3):
        color = []
        color_dict = collections.Counter(image[:, :, i].ravel())
        for num in range(256):
            if not num in color_dict:
                color_dict[num] = 0
        for index in sorted(color_dict.keys()):
            color.append(color_dict[index])
        color_list.append(color)
        #3チャンネルごとの画素数の類型をlistに保存 listの長さ=3
    return color_list

def count_hsv(image):
    hsv_list = []
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    h,s,v = cv2.split(hsv)
    for i, data in enumerate([h, s, v]):
        sort_data = []
        end = 256
        data_dict = collections.Counter(data.ravel())
        if i == 0:
            end = 180
        for num in range(end):
            if not num in data_dict:
                data_dict[num] = 0
        for index in sorted(data_dict.keys()):
            sort_data.append(data_dict[index])
        hsv_list.append(sort_data)
    return hsv_list

def view_color(result_list):
    plt.figure(figsize=(12, 5))
    plt.plot(result_list[0], color='red', label='Red')
    plt.plot(result_list[1], color='green', label='Green')
    plt.plot(reslut_list[2], color='blue', label='Blue')
    plt.legend()
    plt.show()
    
# 深度情報の特徴量を作成する関数
def get_depth_feature(img):
    depth_dict = {}
    depth_list = []
    MAX_DEPTH = 400
    for col in img:
        for num in col:
            if num not in depth_dict:
                depth_dict[num] = 1
            else:
                depth_dict[num] += 1
    
    for num in range(50, MAX_DEPTH):
        if num not in depth_dict:
            depth_list.append(0)
        else:
            depth_list.append(depth_dict[num])
    
    return depth_list


# 色と深度情報を合わせた特徴量のデータフレームを返す関数
def color_and_depth(color_img, depth_img, name):
    result_dict = {}
    RGB = list(itertools.chain.from_iterable(count_color(color_img)))
    HSV = list(itertools.chain.from_iterable(count_hsv(color_img)))
    Depth = get_depth_feature(depth_img)
    features = RGB + HSV + Depth
    result_dict[name] = features
    df = pd.DataFrame.from_dict(result_dict, orient='index')
    return df

# 深度情報の特徴量の確認

In [ ]:
depth_img = cv2.imread('data/')